# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [ ]:
import textdistance
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [ ]:
corpus = open('sample_data/correct_sents.txt', encoding='utf8').read()

In [ ]:
vocab = Counter(re.findall('\w+', corpus.lower()))

In [ ]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [ ]:
N = sum(vocab.values())

def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(topn)


def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def P(word, N=N):
    return vocab[word] / N

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates] 
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    var = []
    distances = [el[1] for el in closest] # выбиарем наиболее вероятное слово, если у двух вариантов одинаково большой показатель
    if distances[0] == distances[1]:
        for i in closest:
        var.append(P(i[0]))
    else: 
        return closest[0]
    max_var = var.index(max(var))
    
    return closest[max_var]

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [ ]:
get_closest_hybrid_match('клб', X, vec)

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [ ]:
# ваш код тут

In [ ]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in vocab)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
correction('ссоолнце')

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

Byte-Pair Encoding - алгоритм, который используется для сжатия данных в корпусе. Суть алгоритма в том, что он разбивает слова на отдельные буквы (=токены), после чего выбирает токены, которые чаще всего употребляются вместе. Эти сочетания алгоритм объединяет в один новый токен и с их помощью заново кодирует текст. 
1) в ходе обучения алгоритм определяет наиболее популярные сочетания букв-токенов. Сначала алгоритм считает все пары символов, которые часто употребляются вместе (пример - англ. est_, er_), после чего преобразует эти две буквы в один символ. 

2) На тест-данных алгоритм разбивает слова на токены и кодирует символы, которые на предыдущем этапе определил как часто употребляющиеся вместе. Проходит по всем токенам по убыванию частоты их появления в корпусе. 
Алгоритм работает до тех пор, пока не обработаются все токены, или пока не кончится заданное число итераций. 
Дополнительно используется символ (стоп-токен), который разграничевает слова в корпусе. Благодаря этому алгоритм может определить позицию сочетания токенов в слове (estimate и highest_ - во втором случае будет связка est_  - указание на то, что она находится в конце слова).

На выходе получается перекодированный корпус, в котором несколько токенов могут быть закодированы одним общим, благодаря чему он занимает меньше памяти.